# Download Libraries

In [ ]:
!pip install ultralytics

In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import yaml
from ultralytics import YOLO
import pandas as pd
import json

# Changing Data to YOLO Format & Data Augmentation
My method is duplicating to x2 the number of labels in Banana, Pineapple, Watermelon Object. And about Orange Object i just take a hafl of it's original data. So you can see at Below Section: Count labels per... I have shown all the number of labels per Object after processing data.

Processing file can find at :  https://www.kaggle.com/datasets/bithlong/processing-data  
Best.pt can find at         :  https://www.kaggle.com/datasets/bithlong/trained-yolo-model  
Final Dataset can be find at:  https://www.kaggle.com/datasets/bithlong/final-pro-dataset  


# Split Train and Val Dataset

In [ ]:
src_images_dir = '/kaggle/input/final-pro-dataset/yolo_dataset/new_train/images'
src_labels_dir = '/kaggle/input/final-pro-dataset/yolo_dataset/new_train/labels'

images = [f for f in os.listdir(src_images_dir) if f.endswith('.jpg')]
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

# Create new train/val directories
os.makedirs('/kaggle/working/yolo_dataset/train/images', exist_ok=True)
os.makedirs('/kaggle/working/yolo_dataset/train/labels', exist_ok=True)
os.makedirs('/kaggle/working/yolo_dataset/val/images', exist_ok=True)
os.makedirs('/kaggle/working/yolo_dataset/val/labels', exist_ok=True)

# Copy files
for img in images:
    shutil.copy(f'/kaggle/input/final-pro-dataset/yolo_dataset/new_train/images/{img}', f'/kaggle/working/yolo_dataset/train/images/{img}')
    shutil.copy(f'/kaggle/input/final-pro-dataset/yolo_dataset/new_train/labels/{img.replace(".jpg", ".txt")}', f'/kaggle/working/yolo_dataset/train/labels/{img.replace(".jpg", ".txt")}')
for img in val_imgs:
    shutil.copy(f'/kaggle/input/final-pro-dataset/yolo_dataset/new_train/images/{img}', f'/kaggle/working/yolo_dataset/val/images/{img}')
    shutil.copy(f'/kaggle/input/final-pro-dataset/yolo_dataset/new_train/labels/{img.replace(".jpg", ".txt")}', f'/kaggle/working/yolo_dataset/val/labels/{img.replace(".jpg", ".txt")}')

# Create .ymal Configuration File

In [ ]:
data_yaml = {
    'train': '/kaggle/working/yolo_dataset/train/images',
    'val' : '/kaggle/working/yolo_dataset/val/images',
    'test': '/kaggle/input/final-pro-dataset/yolo_dataset/test/images',
    'nc': 6,
    'names': {
        0: "Apple",
        1: "Banana",
        2: "Grapes",
        3: "Orange",
        4: "Pineapple",
        5: "Watermelon"
    }
}

with open('/kaggle/working/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

# Verify
!cat /kaggle/working/data.yaml

# Count labels per class to check for imbalance:

Training Dataset

In [ ]:
# import os
# from collections import Counter

# train_class_counts = Counter()
# for label_file in os.listdir('/kaggle/input/final-pro-dataset/yolo_dataset/new_train/labels'):
#     with open(f'/kaggle/input/final-pro-dataset/yolo_dataset/new_train/labels/{label_file}') as f:
#         for line in f:
#             class_id = line.split()[0]
#             train_class_counts[class_id] += 1
# print(train_class_counts)
# # Counter({'3': 7146, '0': 6771, '2': 6605, '1': 6332, '5': 3440, '4': 2838})

# Training Model
The model that i used is YOLO V11 Medium

In [ ]:
# Load pre-trained model
model = YOLO('yolo11m.pt') # 

# Train
model.train(
    # === BASIC SETTINGS ===
    data='/kaggle/working/data.yaml',
    epochs=100,              # Increased for larger dataset
    imgsz=640,
    batch=16,                # Increased batch size for better gradient estimates
    device=0,
    
    # === LEARNING RATE & OPTIMIZATION ===
    lr0=0.0005,              # Lower initial learning rate for stability
    lrf=0.001,               # Final learning rate
    momentum=0.937,         # Standard momentum
    weight_decay=0.0005,    # L2 regularization
    warmup_epochs=5,        # More warmup for larger dataset
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    optimizer='AdamW',
    
    # === LOSS WEIGHTS (adjusted for class imbalance) ===
    box=0.1,                # Bounding box loss weight
    cls=0.7,                # REDUCED classification loss (you have good balance now)
    dfl=1.5,                # Distribution focal loss
    
    # # === DATA AUGMENTATION (optimized for 19k images) ===
    # # Geometric augmentations
    # degrees=15,             # Reduced rotation (fruits have natural orientation)
    # translate=0.1,          # Small translation
    # scale=0.8,              # More aggressive scaling
    # shear=3.0,              # Shear transformation
    # perspective=0.0001,     # Very small perspective change
    
    # Flip augmentations
    flipud=0.0,             # NO vertical flip (fruits orientation matters)
    fliplr=0.0,             # Keep horizontal flip
    
    # Advanced augmentations
    # mosaic=1.0,             # Full mosaic (great for detection)
    mixup=0.15,              # Light mixup for robustness
    copy_paste=0.2,         # Copy-paste augmentation
    
    # # Color augmentations (important for fruits!)
    # hsv_h=0.008,            # Subtle hue changes
    # hsv_s=0.6,              # Moderate saturation changes
    # hsv_v=0.3,              # Brightness variations
    
    # === TRAINING STRATEGY ===
    patience=25,            # Early stopping patience
    save_period=10,         # Save every 10 epochs
    val=True,               # Enable validation
    plots=True,             # Generate training plots
    
    # === ADVANCED SETTINGS ===
    amp=True,               # Automatic Mixed Precision (faster training)
    fraction=1.0,           # Use full dataset
    profile=False,          # Set True for performance profiling
    
    # === OUTPUT ===
    name='fruit_detection_balanced_v2',
    exist_ok=True,
    
    # === ADDITIONAL OPTIMIZATIONS ===
    close_mosaic=10,        # Stop mosaic in last 10 epochs for better final training
    resume=False,           # Set to checkpoint path if resuming
)

# Test Model on Validation Dataset

In [ ]:
results = model.val()
print(f"mAP@0.5: {results.box.map}")
for i, ap in enumerate(results.box.ap):
    print(f"AP@0.5 for {results.names[i]}: {ap}")

# Generate Prediction on Test Dataset & Save to Submission.csv


The result on 30% is 0.76 which is pretty low comparing to mAP the model have shown in training process. Which mean the model is so-called overfitted

In [ ]:
def generate_submission_csv(model_path, test_images_dir, submission_path="submission.csv"):
    """
    Tạo file submission 
    
    Args:
        model_path (str): Đường dẫn đến file weight của mô hình YOLO.
        test_images_dir (str): Thư mục chứa ảnh test.
        submission_path (str): Đường dẫn lưu file CSV output.
    """
    model = YOLO(model_path)
    image_files = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    submission_data = []

    for image_file in image_files:
        img_path = os.path.join(test_images_dir, image_file)

        result = model(img_path, conf=0.1, iou=0.5, verbose=False)[0]

        predictions = []
        for box in result.boxes:
            xyxy = box.xyxy[0].tolist()
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            w = xyxy[2] - xyxy[0]
            h = xyxy[3] - xyxy[1]

            if conf >= 0.01 and w * h > 50:  
                predictions.append({
                    "x_min": round(xyxy[0], 2),
                    "y_min": round(xyxy[1], 2),
                    "x_max": round(xyxy[2], 2),
                    "y_max": round(xyxy[3], 2),
                    "class": cls_id,
                    "confidence": round(conf, 3)
                })

        image_id = os.path.splitext(image_file)[0]
        formatted_id = f"img{int(image_id)}" if image_id.isdigit() else image_id

        submission_data.append({
            "ID": formatted_id,
            "bounding_boxes": json.dumps(predictions)
        })

    df = pd.DataFrame(submission_data)
    df.to_csv(submission_path, index=False)
    print(f"Submission file saved to {submission_path}")


generate_submission_csv(
    model_path="/kaggle/input/trained-yolo-model/best.pt",
    test_images_dir="/kaggle/input/final-pro-dataset/yolo_dataset/test/images",
    submission_path="/kaggle/working/submission.csv"
)